# Frequent Pattern Mining

## Introduction
Frequent Pattern Mining (FPM) identifies common co-occurring words or patterns within a dataset. For this project, we applied FPM to analyze a collection of tweets, aiming to uncover recurring themes related to emergencies.

## Preprocessing
Before running the FPM algorithm, we cleaned and prepared the dataset:
1. **Cleaned the Text**: Removed punctuation, URLs, and stopwords to focus on meaningful words.
2. **Handled Missing Data**: Replaced missing values in the `keyword` and `location` columns with empty strings.
3. **Created Transactions**: Each tweet was tokenized into words, and the corresponding `keyword` and `location` were added to form transactions.

In [31]:
! pip install nltk
! pip install mlxtend

/usr/local/python/3.12.1/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=5658) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


/usr/local/python/3.12.1/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=5658) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [32]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [33]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
data = pd.read_csv('tweetsv2.csv')
# Function to clean text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Apply text cleaning
data['cleaned_text'] = data['text'].apply(clean_text)
data['tokens'] = data['cleaned_text'].apply(lambda x: x.split())

# Include keywords and locations in transactions
data['keyword'] = data['keyword'].fillna('').astype(str)
data['location'] = data['location'].fillna('').astype(str)
data['transactions'] = data.apply(lambda row: row['tokens'] + [row['keyword'], row['location']], axis=1)

# Verify cleaned transactions
print(data['transactions'].head())


0    [communal, violence, bhainsa, telangana, stone...
1    [telangana, section, imposed, bhainsa, january...
2    [arsonist, sets, cars, ablaze, dealership, abl...
3    [arsonist, sets, cars, ablaze, dealership, abl...
4    [lord, jesus, love, brings, freedom, pardon, f...
Name: transactions, dtype: object


## Frequent Itemsets
Using the Apriori algorithm, we extracted frequent itemsets with a minimum support of 1%. Frequent itemsets represent words or combinations of words that appear together in at least 1% of the transactions.

In [35]:
# Transaction Encoding
transactions = data['transactions'].tolist()
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Apply Apriori
min_support = 0.01  # Set minimum support
frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)

# Display frequent itemsets
print("Frequent Itemsets:")
print(frequent_itemsets.head())

Frequent Itemsets:
    support   itemsets
0  0.300616         ()
1  0.010026      (air)
2  0.013369     (also)
3  0.044943      (amp)
4  0.010026  (another)


## Association Rules
Association rules were generated from the frequent itemsets to find relationships between co-occurring words. These rules show patterns where the presence of one word predicts another with high confidence.

In [36]:
# Generate Association Rules
min_confidence = 0.25
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence, num_itemsets=len(frequent_itemsets))
# Display rules
print("Association Rules:")
print(rules[['antecedents', 'consequents', 'confidence', 'lift']])

Association Rules:
  antecedents consequents  confidence      lift
0       (amp)          ()    0.309198  1.028548
1        (im)          ()    0.284689  0.947020
2      (like)          ()    0.290258  0.965547
3       (one)          ()    0.305000  1.014585
4    (people)          ()    0.306163  1.018453


## Analysis and Limitations
- Frequent Itemsets: Patterns like ['fire', 'rescue'] indicate common co-occurrences in emergency-related tweets.
- Sparse Data: Tweets are short, so many patterns consist of single words rather than multi-word itemsets.
- Association Rules: While some meaningful rules were generated, many transactions lacked strong co-occurrences.

## Future Work
N-Grams: Use bigrams or trigrams to extract richer context from tweets.
Lower Thresholds: Experiment with even lower support and confidence thresholds to uncover less frequent patterns.
Alternative Methods: Explore clustering or classification to complement frequent pattern analysis.